<a href="https://colab.research.google.com/github/michalcohen/DataAnalysisABC/blob/main/find_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# installs
!pip install jupyter-dash

In [19]:
# imports
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash
import dash_cytoscape as cyto
import urllib.request, json

In [ ]:
import graphistry
graphistry.register(api=3, username='michal', password='dbTrw7BDuQGZ@CF')

In [21]:
# Load Data
df = px.data.tips()
with urllib.request.urlopen("https://raw.githubusercontent.com/michalcohen/DataAnalysisABC/main/pass1.json") as url:
    data = json.loads(url.read().decode())
elements = data["nodes"] + data["edges"]

In [22]:
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
                       html.P("Dash Cytoscape:"),
                       cyto.Cytoscape(id='cytoscape',
                                      elements=elements,
                                      layout={'name': 'breadthfirst'},
                                      style={'width': '800px', 'height': '1000px'}
                                      )
                       ])
app.run_server(mode='inline')

<IPython.core.display.Javascript object>